In [93]:
import tensorflow as tf

In [94]:
import numpy as np

# 2.1.1常量张量

In [95]:
# 张量即多为数组， tensorflow的数据结构即为张量
a = tf.constant(1) # tf.int32 类型常量
b = tf.constant(1.23) #tf.float32 类型常量
c = tf.constant("hello world") # tf.string类型常量
d=  tf.constant(True) #tf.bool类型常量

In [96]:
print(a,b,c,d, sep='\n')

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1.23, shape=(), dtype=float32)
tf.Tensor(b'hello world', shape=(), dtype=string)
tf.Tensor(True, shape=(), dtype=bool)


In [97]:
print(tf.rank(a))  # 0维张量

tf.Tensor(0, shape=(), dtype=int32)


In [98]:
# [] 中括号的数量代表张量的维度
vector_1 = tf.constant([1.0,2.0,3.0,4.0]) #向量，1维张量
vector_2 = tf.constant([[1,2],[3,4]])     # 2维张量
vector_3 = tf.constant([[[1,2],[3,4]],[[1,2],[3,4]]])  # 3维张量

In [99]:
print(tf.rank(vector_1), tf.rank(vector_2),tf.rank(vector_3),'\n' ,vector_1.shape, vector_2.shape, vector_3.shape)

tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(2, shape=(), dtype=int32) tf.Tensor(3, shape=(), dtype=int32) 
 (4,) (2, 2) (2, 2, 2)


In [100]:
print(vector_1, vector_2, vector_3, sep='\n')

tf.Tensor([1. 2. 3. 4.], shape=(4,), dtype=float32)
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[[1 2]
  [3 4]]

 [[1 2]
  [3 4]]], shape=(2, 2, 2), dtype=int32)


In [101]:
v_3_float = tf.cast(vector_3, tf.float32)   # tf.cast() 改变张量的数据类型

In [102]:
print(v_3_float,'\n',vector_3 )

tf.Tensor(
[[[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]], shape=(2, 2, 2), dtype=float32) 
 tf.Tensor(
[[[1 2]
  [3 4]]

 [[1 2]
  [3 4]]], shape=(2, 2, 2), dtype=int32)


In [103]:
# vector_4= vector_2+tf.constant([[1.0,1.0],[1.0,1.0]])   # 不同整数类型无法相加
vector_2 = tf.cast(vector_2, tf.float32)  
vector_4= vector_2+tf.constant([[1.0,1.0],[1.0,1.0]])

In [104]:
print(vector_4)

tf.Tensor(
[[2. 3.]
 [4. 5.]], shape=(2, 2), dtype=float32)


# 2.1.2模型中被训练的参数设置为变量 变量张量

In [105]:
a = tf.Variable([1.0,2.0],name = "v")
b = tf.Variable([1.0, 2])

In [106]:
print(a,b, id(a) ,id(b),sep='\n')

<tf.Variable 'v:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
1626276329544
1626280897608


# 2.2 三种计算图

在TensorFlow1.0时代，采用的是静态计算图，需要先使用TensorFlow的各种算子创建计算图，然后再开启一个会话Session，显式执行计算图
TensorFlow2.0时代，采用的是动态计算图，即每使用一个算子后，该算子会被动态加入到隐含的默认计算图中立即执行得到结果，而无需开启Session。Eager Excution的好处是方便调试程序，它会让TensorFlow代码的表现和Python原生代码的表现一样，写起来就像写numpy一样，各种日志打印，控制流全部都是可以使用的。
TensorFlow2.0中使用静态图，可以使用@tf.function装饰器将普通Python函数转换成对应的TensorFlow计算图构建代码。运行该函数就相当于在TensorFlow1.0中用Session执行代码。使用tf.function构建静态图的方式叫做 Autograph
计算图由节点(nodes)和线(edges)组成。

节点表示操作符Operator，或者称之为算子，线表示计算间的依赖。

实线表示有数据传递依赖，传递的数据即张量。

虚线通常可以表示控制依赖，即执行先后顺序。

# 2.2.1 静态计算图

In [107]:
# Tensorflow 2.0  使用静态计算图方法
g = tf.compat.v1.Graph()
with g.as_default():
    #placeholder为占位符，执行会话时候指定填充对象
    x = tf.compat.v1.placeholder(name='x', shape=[], dtype=tf.string)
    y = tf.compat.v1.placeholder(name='y', shape=[], dtype=tf.string)
    z = tf.strings.join([x,y],name = "join",separator = " ")

with tf.compat.v1.Session(graph = g) as link:
    # fetches的结果非常像一个函数的返回值，而feed_dict中的占位符相当于函数的参数序列。
    result = link.run(fetches = z,feed_dict = {x:"hello",y:"good"})
    print(result)

b'hello good'


In [108]:
# 在TensorFlow2.0中，使用的是动态计算图和Autograph，赋值之后即可立即使用
x = tf.constant('hello')
y = tf.constant('good')
z = tf.strings.join([x,y], separator="")

In [109]:
print(tf.rank(x))  # 对应之前，没有中括号，0维张量
print(z)
tf.print(z)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(b'hellogood', shape=(), dtype=string)
hellogood


In [110]:
#  封装成函数使用
def joinstr(x,y):
    z = tf.strings.join([x,y], separator="")
    tf.print(z)
    return z

ans = joinstr(x,y)   # x,y 之前已经定义，运行后可以直接使用
print(ans)   # 函数化后效果类似

hellogood
tf.Tensor(b'hellogood', shape=(), dtype=string)


 实践中，我们一般会先用动态计算图调试代码，然后在需要提高性能的的地方利用@tf.function切换成Autograph获得更高的效率

In [111]:
# @tf.function装饰器将普通Python函数转换成和TensorFlow1.0对应的静态计算图构建代码。

@tf.function
def strjoin(x,y):
    z =  tf.strings.join([x,y],separator = " ")
    tf.print(z)
    return z

result = strjoin(x,y)
print(result)

hello good
tf.Tensor(b'hello good', shape=(), dtype=string)


In [112]:
import datetime

# 创建日志
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = './data/autograph/%s' % stamp
writer = tf.summary.create_file_writer(logdir)
#开启autograph跟踪
tf.summary.trace_on(graph=True, profiler=True) 
#执行autograph
result = strjoin("hello","world")

hello world


In [113]:
with writer.as_default():
    tf.summary.trace_export(
        name="autograph",
        step=0,
        profiler_outdir=autograph)

NameError: name 'autograph' is not defined

In [ ]:
%load_ext tensorboard

In [ ]:
#启动tensorboard
%tensorboard --logdir ./data/autograph/

# 2.3 自动微分机制
Tensorflow一般使用梯度磁带tf.GradientTape来记录正向运算过程，然后反播磁带自动得到梯度值。这种利用tf.GradientTape求微分的方法叫做Tensorflow的自动微分机制。

In [115]:
# f(x) = a*x**2 + b*x + c的导数  a=1,b=-3,c=1
x = tf.Variable(0.0, name='x', dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-3.0)
c = tf.constant(1.0)

with tf.GradientTape() as tape:
    y = a*tf.pow(x,2) + b*x + c
    
dy_dx = tape.gradient(y, x)
print(dy_dx)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
tf.Tensor(-3.0, shape=(), dtype=float32)


In [117]:
# f(x) = a*x**2 + b*x + c的导数  a=1,b=-3,c=1
# 对常量张量求导，增加watch
with tf.GradientTape() as tape:
    tape.watch([a,b,c])
    y = a*tf.pow(x,2) + b*x + c
    
dy_dx,dy_da,dy_db,dy_dc = tape.gradient(y,[x,a,b,c])
print(dy_dx,dy_da,dy_db,dy_dc, sep='\n')   # 结果显示，x 默认为0

tf.Tensor(-3.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


untimeError: GradientTape.gradient can only be called once on non-persistent tapes.
 GradientTape.gradient 每次使用时都要调用

In [119]:
# f(x) = a*x**2 + b*x + c的导数  a=1,b=-3,c=1 求二阶导数
with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:   
        y = a*tf.pow(x,2) + b*x + c
    dy_dx = tape1.gradient(y,x)   

dy2_dx2 = tape2.gradient(dy_dx, x)
print(dy2_dx2)

tf.Tensor(2.0, shape=(), dtype=float32)


In [123]:
# 再 autograph 中使用
@tf.function
def f(x):
    a, b, c = tf.constant(1.0), tf.constant(-2.0), tf.constant(1.0)
    x = tf.cast(x, tf.float32)
    
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = a*tf.pow(x,2) + b*x + c
    dy_dx = tape.gradient(y, x)
    return (dy_dx, y)

tf.print(f(tf.constant(0.0)))
tf.print(f(tf.Variable(1.0)))

(-2, 1)
(0, 0)


# 2.3.2 利用梯度磁带和优化器求最小值
求f(x) = a*x**2 + b*x + c的最小值,使用optimizer.apply_gradients
  实际函数参数赋值之后：y = x^2 - 2x +1

In [133]:
x = tf.Variable(0.0,name = "x",dtype = tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)  # 
for _ in range(1000):
    with tf.GradientTape() as tape:
        y = a*tf.pow(x,2) + b*x + c
    dy_dx = tape.gradient(y,x)
    optimizer.apply_gradients(grads_and_vars=[(dy_dx,x)])

tf.print("y= ", y, "; x= ", x)

y=  0 ; x=  0.999998569


In [138]:
# 使用optimizer.minimize
# optimizer.minimize相当于先用tape求gradient,再apply_gradient

x = tf.Variable(0.0,name = "x",dtype = tf.float32)

#注意f()无参数
def f():   
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    y = a*tf.pow(x,2)+b*x+c
    return(y)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.05)   # 学习率
for _ in range(150):
    optimizer.minimize(f,[x])   

tf.print("y =",f(),"; x =",x)

y = 0 ; x = 0.999999762


In [141]:
# 使用autograph 回溯版本报错，大致掌握以上两张即可